In [ ]:
pip install rdflib

In [ ]:
pip install dateparser

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
import pandas as pd #read xslx
from rdflib import Graph, Literal, RDF, URIRef, Namespace , BNode
from rdflib.namespace import FOAF , XSD, DC, FOAF, SKOS, RDF, RDFS
import urllib.parse 
from google.colab import files 
import io
from datetime import datetime
import dateparser
from dateparser.search import search_dates


# Input files

In [ ]:
uploaded = files.upload()

# Helper functions

In [ ]:
import hashlib
import uuid

def concept_uri(base_uri, input):
  m = hashlib.md5()
  m.update(input.encode('utf-8'))

  return URIRef(base_uri + m.hexdigest())

def addLiteral(subject, predicate, objectColumn, datatype=None):
  if pd.notna(row[objectColumn]):
    if datatype == None:
      g.add((subject, predicate, Literal(str(row[objectColumn]), lang='nl')))
    else:
      g.add((subject, predicate, Literal(str(row[objectColumn]), datatype=datatype)))

# using statbel firstnames of newborns (heuristic)
m = pd.read_excel('Voornamen_Jongens_1995-2017_0.xls', sheet_name='1995-2019')
male_names = (m['Unnamed: 1'].append(m['Unnamed: 4']).append(m['Unnamed: 7']).append(m['Unnamed: 10'])).unique()
f = pd.read_excel('Voornamen_meisjes_1995-2017.xls', sheet_name='1995-2019')
female_names = (f['Unnamed: 1'].append(f['Unnamed: 4']).append(f['Unnamed: 7']).append(f['Unnamed: 10'])).unique()

manual_entries = ['Friedo', 'Renilde', 'Jozef', 'Maria-André', 'Gedo', 'Yvo', 'Marie-Cecile', 'Fonny', 'Luciaan', 'Willy', 'Fredy']
likely_last_names = ['Vos', 'Matthijs', 'Stevens', 'Maere', 'Rubens', 'Beer', 'Duran', 'Roos', 'Broos', 'Thijs', 'Perre', 'Joris', 'Winter', 'Claus', 'Thys', 'Massa', 'Roy']

first_names = np.concatenate([male_names,female_names, manual_entries])

first_names = np.delete(first_names, np.where(first_names == 'Van'))
first_names = np.delete(first_names, np.where(first_names == 'Blomme'))


def is_known_first_name(potential_name):
  return potential_name in first_names


def splitname(full_name):
  full_name.str.split(' ')


In [ ]:
import re

mv = re.compile('(mevr|dhr)[\.]?[\s]?', re.IGNORECASE)

def remove_title(full_name):
  return mv.sub('', full_name)


def splitname(full_name):
  first = last = ''
  comment = []

  split = remove_title(full_name).split(' ')

  if len(split) == 1 : comment.append('Cannot split name')

  potential_first_last = is_known_first_name(split[0])
  potential_last_first = is_known_first_name(split[-1])

  if potential_first_last and potential_last_first:
    if split[-1] in likely_last_names:
      first = split[0]
      last = ' '.join(split[1:])
    elif split[0] in likely_last_names:
      first = split[-1]
      last = ' '.join(split[0:-1])
    comment.append('Ambiguous: two possible first names - {}'.format(full_name))
  elif potential_first_last:
    first = split[0]
    last = ' '.join(split[1:])
  elif potential_last_first:
    first = split[-1]
    last = ' '.join(split[0:-1])
  else:
    comment.append('No potential first name found - {}'.format(full_name))
    # print([full_name])
  return [first, last, ' - '.join(comment)]

## split_house_bus_number

In [ ]:
def split_house_bus_number(house_bus_number):
  house_number = bus_number = np.NaN
  comment = []
  house_bus_number = house_bus_number.replace(' ', '')

  if ('z/n' not in house_bus_number and 'nan' not in house_bus_number) : 
    if (('bus' in house_bus_number or '/' in house_bus_number)) :
      comment.append('Splitting. Check it.')
      if ('bus' in house_bus_number) : 
        split = house_bus_number.split('bus')
      else :
        split = house_bus_number.split('/')
      house_number = split[0]
      bus_number = split[1]
    else:
        house_number = house_bus_number
    house_number =  house_number.replace('/', '').replace('-', '').replace(',', '')
  else :
    comment.append('No house nr found')
      
  return [house_number, bus_number, ' - '.join(comment)]

### kbo_cleansing

In [ ]:
def kbo_cleansing(kbo):
  kbo_cleansed = comment = np.NaN

  if kbo != 'nan':
    kbo = re.sub(r'\D', '', kbo)
    if  re.match(r'\d{10}', kbo):
      kbo_cleansed = kbo
    elif re.match(r'\d{9}', kbo):
      kbo_cleansed = kbo
      comment = 'only 9 digits. Missing first 0?'
    else: 
      comment = 'Wrong KBO format. Check it.'
  else :
    comment = 'No KBO nr found'

  return [kbo_cleansed, comment]

## mail_cleansing

In [ ]:
def mail_cleansing(mail):
  mail_cleansed = comment = np.NaN

  if mail != 'nan':
    if  re.match(r'[\w\.-]+@[\w\.-]+(\.[\w]+)+', mail):
      mail_cleansed = mail
    else: 
      comment = 'Wrong mail format. Check it.'
  else:
    comment = 'No mail found'

  return [mail_cleansed, comment]

def find_all_mails(input):
  mails = input.findall(r'[\w\.-]+@[\w\.-]+')



##telephone_number_cleansing

In [ ]:
def telephone_number_cleansing(telephone_number):
  telephone_number = re.sub(r'\s', '', telephone_number)

  telephone_number = re.sub(r'tel:', '', telephone_number)
  telephone_number = re.sub(r'tel', '', telephone_number)
  telephone_number = re.sub(r'<br>', '', telephone_number)

  telephone_number = re.sub(r'^\+32-\(0\)', '0', telephone_number)
  telephone_number = re.sub(r'^\+32', '0', telephone_number)
  telephone_number = re.sub(r'^32', '0', telephone_number)

  telephone_number = re.sub(r'^\+0032-\(0\)', '0', telephone_number)
  telephone_number = re.sub(r'^0032-\(0\)', '0', telephone_number)
  telephone_number = re.sub(r'^0032', '0', telephone_number)

  telephone_number = re.sub(r'^\(\d\d\d\)', '0', telephone_number)
  telephone_number = re.sub(r'^0\(\d\d\d\)', '0', telephone_number)

  telephone_number =  re.sub(r'[\.a-zA-Z]', '', telephone_number)

  return split_telephone_number(telephone_number)

def split_telephone_number(telephone_number):
  telephone_number_1 = telephone_number_2 = np.NaN
  comment = []

  split = [telephone_number]
  if '//' in telephone_number:
    split = telephone_number.split('//')
  elif '-' in telephone_number:
    split = telephone_number.split('-')
  elif ';' in telephone_number:
    split = telephone_number.split(';')
  elif 'enGSM:' in telephone_number:
    split = telephone_number.split('enGSM:')


  split[0] = split[0].replace('/', '')
  if check_telephone_number_lenght(split[0]):
    telephone_number_1 = split[0]
  else:
    comment.append('Wrong telephone number lenght. Check it. ' + split[0])

  if len(split) == 2 :
    comment.append('Splitting. Check it.')
    split[1] = split[1].replace('/', '')
    if check_telephone_number_lenght(split[1]):
      telephone_number_2 = split[1]
    else:
      comment.append('Wrong telephone 2 number lenght. Check it.')

  return [telephone_number_1, telephone_number_2, ' - '.join(comment)]

def check_telephone_number_lenght(telephone_number):
  if len(telephone_number) < 9 or len(telephone_number) > 10:
    return False
  else :
    return True

### date_cleansing

In [ ]:
def date_cleansing(date):
    dates_parsed = []

    if date != "nan":
      print(">> " + date)
      
      match = re.findall(r'\d{1,2}.\d{1,2}.\d{2,4}', date)
      if match:
        for m in match:
          date_parsed_match = dateparser.parse(m, settings={'DATE_ORDER': 'DMY'})
          dates_parsed.append(date_parsed_match)

      match = re.findall(r'\d{1,2} \w* \d{2,4}', date)
      if match:
        for m in match:
          date_parsed_match = dateparser.parse(m, settings={'DATE_ORDER': 'DMY'})
          dates_parsed.append(date_parsed_match)

      for item in dates_parsed:
          print("..." + str(item))

    return dates_parsed

# Init graph

In [ ]:
g = Graph();

In [ ]:
#namspaces

#lblod = Namespace('https://contacthub-dev.lblod.info/id/')
lblod = Namespace('https://contacthub-qa.lblod.info/id/')

adms = Namespace('http://www.w3.org/ns/adms#')
adres = Namespace('https://data.vlaanderen.be/ns/adres#')
besluit = Namespace('http://data.vlaanderen.be/ns/besluit#')
foaf = Namespace('http://xmlns.com/foaf/0.1/')
generiek = Namespace('https://data.vlaanderen.be/ns/generiek#')
locn = Namespace('http://www.w3.org/ns/locn#')
mandaat = Namespace('http://data.vlaanderen.be/ns/mandaat#')
organisatie = Namespace('https://data.vlaanderen.be/ns/organisatie#')
org = Namespace('http://www.w3.org/ns/org#')
os = Namespace('https://data.vlaanderen.be/id/concept/organisatiestatus/')
person = Namespace('http://www.w3.org/ns/person#')
persoon = Namespace('https://data.vlaanderen.be/ns/persoon#')
rov = Namespace('http://www.w3.org/ns/regorg#')
schema = Namespace('http://schema.org/')


# Predefined concepts:
centraal_bestuur_van_de_eredienst = URIRef("http://data.vlaanderen.be/id/concept/BestuurseenheidClassificatieCode/f9cac08a-13c1-49da-9bcb-f650b0604054")
bestuur_van_de_eredienst = URIRef("http://data.vlaanderen.be/id/concept/BestuurseenheidClassificatieCode/66ec74fd-8cfc-4e16-99c6-350b35012e86")

# No code found for secretaris and voorzitter? https://data.vlaanderen.be/doc/conceptscheme/BestuursfunctieCode
# TODO: do not use hoedanigheid
# -- 

#voorzitter_concept = URIRef("https://data.vlaanderen.be/doc/concept/hoedanigheid/e4c26f11-c267-46e7-b783-afa5ffdcf3f7")
# aka budgethouder? 
# FIXME: WRONG - budgethouder == penningmeester
#secretaris_concept = URIRef("https://data.vlaanderen.be/id/concept/hoedanigheid/fc5d6eaa-175c-4fad-a174-e3cfb3d9c064")

In [ ]:
ckb = pd.read_excel('SP_DownloadLijst_CentraleKerkbesturen_20210110.xlsx', sheet_name='DATA')

In [ ]:
ckb[['Opmerkingen_CKB', 'Verkiezingen17','Verkiezingen17_Opmerkingen','Verkiezingen2020','Verkiezingen2020_Opmerkingen']]

# Data cleansing

Statusses: map strings to canonical form

'Niet actief - niet van toepassing' and 'Niet actief - Niet van toepassing' --> 'Niet actief - Niet van toepassing'

In [ ]:
ckb['Status_CKB_cleansed'] = ckb['Status_CKB'].replace(to_replace='Niet actief - niet van toepassing', value='Niet actief - Niet van toepassing')

## KBO numbers

In [ ]:
ckb['KBO_CKB_cleansed'] = ckb['KBO_CKB'].str.replace(r'\D', '')

In [ ]:
ckb[['KBO_CKB_cleansed', 'KBO_CKB_comment']] = pd.DataFrame(ckb['KBO_CKB'].astype(str).apply(kbo_cleansing).values.tolist(), columns=['kbo_cleansed','comment'])

In [ ]:
ckb['KBO_CKB_comment'].value_counts()

CKB without KBO entry: **109 records**

1.   List item
2.   List item



In [ ]:
ckb[ckb['KBO_CKB_cleansed'].isnull()]

CKB with wrong? kbo entry (not 10 digits): **13 records**

In [ ]:
ckb[~(ckb['KBO_CKB_cleansed'].str.len() == 10) & (ckb['KBO_CKB_cleansed'].notnull())][['KBO_CKB_cleansed', 'KBO_CKB_comment']]

Base organization ID on KBO nr, if missing (e.g. de facto organization), use Titel

In [ ]:
ckb['organization_id'] = ckb['KBO_CKB_cleansed'].fillna(ckb['Titel'])

In [ ]:
ckb['organization_id']

# *Conclusie*: kbo nrs toevoegen. @boris bezorg lijst
# Lijst ontbrekende straten


## Adressen

Provincies lijken correct

In [ ]:
ckb['Provincie_CKB'].unique()

Geen straat: **15 records** - ook bij operationele ckb

In [ ]:
ckb[ckb['Straat_CKB'].isnull()]

### Huisnr + busnr

In [ ]:
ckb[['Huisnr_CKB_Cleansed', 'Busnummer_CKB_Cleansed', 'Huisnr_CKB_Comment']] = pd.DataFrame(ckb['Huisnr_CKB'].astype(str).apply(split_house_bus_number).values.tolist(), columns=['house_number', 'bus_number', 'comment'])
ckb[ckb['Huisnr_CKB_Comment'].astype(bool)][['Huisnr_CKB','Huisnr_CKB_Cleansed', 'Busnummer_CKB_Cleansed', 'Huisnr_CKB_Comment']]

### Zonder huisnr

## Namen

In [ ]:
ckb['Naam_Voorzitter_CKB_cleansed'] = ckb['Naam_Voorzitter_CKB'].str.replace('<br>', '').str.strip()
ckb['Naam_secretaris_CKB_cleansed'] = ckb['Naam_secretaris_CKB'].str.replace('<br>', '').str.strip()

In [ ]:
ckb[['Naam_Voorzitter_CKB_first', 'Naam_Voorzitter_CKB_last', 'Naam_Voorzitter_CKB_comment']] = pd.DataFrame(ckb['Naam_Voorzitter_CKB_cleansed'].astype(str).apply(splitname).values.tolist(), columns=['first', 'last', 'comment'])

In [ ]:
ckb[['Naam_secretaris_CKB_first', 'Naam_secretaris_CKB_last', 'Naam_secretaris_CKB_comment']] = pd.DataFrame(ckb['Naam_secretaris_CKB_cleansed'].astype(str).apply(splitname).values.tolist(), columns=['first', 'last', 'comment'])

In [ ]:
ckb[['Naam_Voorzitter_CKB_first', 'Naam_Voorzitter_CKB_last', 'Naam_Voorzitter_CKB_comment']] = pd.DataFrame(ckb['Naam_Voorzitter_CKB_cleansed'].astype(str).apply(splitname).values.tolist(), columns=['first', 'last', 'comment'])

In [ ]:
ckb[['Naam_Voorzitter_CKB', 'Naam_Voorzitter_CKB_first', 'Naam_Voorzitter_CKB_last', 'Naam_Voorzitter_CKB_comment', 'Naam_secretaris_CKB', 'Naam_secretaris_CKB_first', 'Naam_secretaris_CKB_last', 'Naam_secretaris_CKB_comment']]

Exceptions when parsing Voorzitter names

In [ ]:
ckb[ckb['Naam_Voorzitter_CKB_comment'].astype(bool)][['Naam_Voorzitter_CKB', 'Naam_Voorzitter_CKB_first', 'Naam_Voorzitter_CKB_last', 'Naam_Voorzitter_CKB_comment']]

Exceptions when parsing secretaris names

In [ ]:
ckb[ckb['Naam_secretaris_CKB_comment'].astype(bool)][['Naam_secretaris_CKB', 'Naam_secretaris_CKB_first', 'Naam_secretaris_CKB_last', 'Naam_secretaris_CKB_comment']]

## Mails

In [ ]:
ckb['Mail_CKB']

In [ ]:
ckb[['Mail_CKB_Cleansed', 'Mail_CKB_Comment']] = pd.DataFrame(ckb['Mail_CKB'].astype(str).apply(mail_cleansing).values.tolist(), columns=['mail_cleansed','comment'])

In [ ]:
#ckb['Mail_CKB'].str.findall(r'[\w\.-]+@[\w\.-]+').head(30)
ckb['Mail_CKB'].head(50)

## Phone numbers

Formats:

*   Voorzitter: xxxxx Secretaris: xxxxx
*   V: xxxxx S: xxxx
*   xxxx (voorzitter) xxxxx (secretaris)
*   xxxx (lastname) xxxx (lastname)



### Heuristic
For both voorzitter and secretaris

Try find phone number in the following order:

1.   `xxx (voorzitter/secretaris)`
2.   `Voorzitter/Secretaris: xxx`
3.   `V/S: xxx`
2.   Any phone number: `xxx`

Caveat: Only first match is kept! (currently)


In [ ]:
ckb['Tel_CKB_voorzitter'] = ckb['Tel_CKB'].str.extract(r'([\d/ ]*)\s*\(\s*voorzitter\s*\)', flags=re.IGNORECASE) \
  .fillna(ckb['Tel_CKB'].str.extract(r'Voorzitter\s*:\s*([\d/ ]*)', flags=re.IGNORECASE)) \
  .fillna(ckb['Tel_CKB'].str.extract(r'V\s*:\s*([\d/ ]*)', flags=re.IGNORECASE)) \
  .fillna(ckb['Tel_CKB'].str.extract(r'([\d/ ]*)', flags=re.IGNORECASE))

In [ ]:
ckb['Tel_CKB_secretaris'] = ckb['Tel_CKB'].str.extract(r'([\d/ ]*)\s*\(\s*secretaris\s*\)', flags=re.IGNORECASE) \
  .fillna(ckb['Tel_CKB'].str.extract(r'Secretaris\s*:\s*([\d/ ]*)', flags=re.IGNORECASE)) \
  .fillna(ckb['Tel_CKB'].str.extract(r'S\s*:\s*([\d/ ]*)', flags=re.IGNORECASE)) \
  .fillna(ckb['Tel_CKB'].str.extract(r'([\d/ ]*)', flags=re.IGNORECASE))

In [ ]:
ckb[['Tel_CKB_voorzitter', 'Tel_CKB_secretaris', 'Tel_CKB']].head(30)

### Voorzitter

In [ ]:
ckb[['Tel_CKB_voorzitter_1', 'Tel_CKB_voorzitter_2', 'Tel_CKB_voorzitter_Comment']] = pd.DataFrame(ckb['Tel_CKB_voorzitter'].astype(str).apply(telephone_number_cleansing).values.tolist(), columns=['telephone_number_1', 'telephone_number_2', 'comment'])

Secretaris

In [ ]:
ckb[['Tel_CKB_secretaris_1', 'Tel_CKB_secretaris_2', 'Tel_CKB_secretaris_Comment']] = pd.DataFrame(ckb['Tel_CKB_secretaris'].astype(str).apply(telephone_number_cleansing).values.tolist(), columns=['telephone_number_1', 'telephone_number_2', 'comment'])

In [ ]:
ckb[(ckb['Tel_CKB_secretaris_Comment'] != '') & (ckb['Tel_CKB_secretaris'].notnull())][['Tel_CKB_secretaris', 'Tel_CKB_secretaris_1', 'Tel_CKB_secretaris_2', 'Tel_CKB_secretaris_Comment']].head(50)

In [ ]:
ckb[['Tel_CKB', 'Tel_CKB_voorzitter_1', 'Tel_CKB_voorzitter_2', 'Tel_CKB_voorzitter_Comment','Tel_CKB_secretaris_1', 'Tel_CKB_secretaris_2', 'Tel_CKB_secretaris_Comment']]

# Statusses

Skos concepts

In [ ]:
for status in ckb['Status_CKB'].unique():
  subject = concept_uri(os, status)
  g.add((subject, RDF.type, SKOS.Concept))
  g.add((subject, SKOS.prefLabel, Literal(status, lang='nl')))
  g.add((subject, SKOS.definition, Literal(status, lang='nl')))
  if(status.startswith('Operationeel') or status.startswith('Actief')) :
    g.add((subject, SKOS.broader, os.actief))
  else:
    g.add((subject, SKOS.broader, os.nietactief))
 

In [ ]:
os.nietactief

In [ ]:
ckb['Status_CKB'].unique()

Bestuursfunctiecodes: Voorzitter, secretaris

In [ ]:
voorzitter_concept = concept_uri(lblod + 'concept/BestuursFunctieCode/', 'voorzitter')
  
g.add((voorzitter_concept, RDF.type, SKOS.Concept))
g.add((voorzitter_concept, SKOS.prefLabel, Literal('voorzitter', lang='nl')))
g.add((voorzitter_concept, SKOS.definition, Literal('voorzitter', lang='nl')))
g.add((voorzitter_concept, SKOS.inScheme, URIRef('http://data.vlaanderen.be/id/conceptscheme/BestuursfunctieCode')))

secretaris_concept = concept_uri(lblod + 'concept/BestuursFunctieCode/', 'secretaris')

g.add((secretaris_concept, RDF.type, SKOS.Concept))
g.add((secretaris_concept, SKOS.prefLabel, Literal('secretaris', lang='nl')))
g.add((secretaris_concept, SKOS.definition, Literal('secretaris', lang='nl')))
g.add((secretaris_concept, SKOS.inScheme, URIRef('http://data.vlaanderen.be/id/conceptscheme/BestuursfunctieCode')))


penning_meester_concept = concept_uri(lblod + 'concept/BestuursFunctieCode/', 'penningmeester')

g.add((penning_meester_concept, RDF.type, SKOS.Concept))
g.add((penning_meester_concept, SKOS.prefLabel, Literal('penningmeester', lang='nl')))
g.add((penning_meester_concept, SKOS.definition, Literal('penningmeester', lang='nl')))
g.add((penning_meester_concept, SKOS.inScheme, URIRef('http://data.vlaanderen.be/id/conceptscheme/BestuursfunctieCode')))


# Uniformiseren cf eredienstbesturen

## Verkiezingen

In [ ]:
ckb[['Verkiezingen17', 'Verkiezingen17_Opmerkingen', 'Verkiezingen2020', 'Verkiezingen2020_Opmerkingen' ]]

In [ ]:
ckb['Verkiezingen17'].dropna().unique()

In [ ]:
ckb['Verkiezingen17_Opmerkingen'].dropna().unique()

In [ ]:
  for index, row in ckb.iterrows():
    date = str(row['Verkiezingen17_Opmerkingen'])

    if date != 'nan':
      dates_parsed = date_cleansing(date)

      if dates_parsed :
        ckb.at[index, 'Verkiezingen17_Opmerkingen Cleansed'] = dates_parsed[0]
        if len(dates_parsed) > 1:
          comment = []
          for i in range(1, len(dates_parsed)):
            comment.append(str(dates_parsed[i]))
            ckb.at[index, 'Verkiezingen17_Opmerkingen Comment'] = ' - '.join(comment)
        else:
          ckb.at[index, 'Verkiezingen17_Opmerkingen Comment'] = np.NaN
      else:
        ckb.at[index, 'Verkiezingen17_Opmerkingen Cleansed'] = np.NaN

        ckb['Verkiezingen17_Opmerkingen Comment'] = ckb['Verkiezingen17_Opmerkingen Comment'].astype(object)
        ckb.at[index, 'Verkiezingen17_Opmerkingen Comment'] = 'Wrong date format. Check it.'


In [ ]:
ckb['Verkiezingen17_Opmerkingen Comment'].dropna().unique()

In [ ]:
ckb['Verkiezingen17_Opmerkingen Cleansed'].dropna().unique()

In [ ]:
ckb[ckb['Verkiezingen17_Opmerkingen Comment'].str.contains('\w', na=False)]

In [ ]:
ckb['Verkiezingen2020_Opmerkingen'].dropna().unique()

In [ ]:
  for index, row in ckb.iterrows():
    date = str(row['Verkiezingen2020_Opmerkingen'])

    if date != "nan":
      dates_parsed = date_cleansing(date)
      if dates_parsed :
        ckb.at[index, 'Verkiezingen2020_Opmerkingen Cleansed'] = dates_parsed[0]
        if len(dates_parsed) > 1:
          comment = []
          for i in range(1, len(dates_parsed)):
            comment.append(str(dates_parsed[i]))
            ckb.at[index, 'Verkiezingen2020_Opmerkingen Comment'] = ' - '.join(comment)
        else:
          ckb.at[index, 'Verkiezingen2020_Opmerkingen Comment'] = np.NaN
      else:
        ckb.at[index, 'Verkiezingen2020_Opmerkingen Cleansed'] = np.NaN

        ckb['Verkiezingen2020_Opmerkingen Comment'] = ckb['Verkiezingen2020_Opmerkingen Comment'].astype(object)
        ckb.at[index, 'Verkiezingen2020_Opmerkingen Comment'] = 'Wrong date format. Check it.'


In [ ]:
ckb['Verkiezingen2020_Opmerkingen Comment'].dropna().unique()

In [ ]:
ckb['Verkiezingen2020_Opmerkingen Cleansed'].dropna().unique()

#Mapping

In [ ]:
ckb.info()

In [ ]:
# TODO: correct uri generation (more random)
for index, row in ckb.iterrows():
  identifier = row['organization_id'] # Should become KBO! 
  # ABB_URI
  s = concept_uri(lblod + 'organisatie/', str(row['organization_id']))
  g.add((s, RDF.type, org.Organization))

  addLiteral(s, SKOS.prefLabel, 'Naam_CKB', datatype=XSD.string)
  addLiteral(s, rov.legalName, 'Naam_CKB', datatype=XSD.string)

  kbo_uri = concept_uri(lblod + 'gestructureerdeIdentificator/', str(row['KBO_CKB_cleansed']))
  g.add((kbo_uri, RDF.type, generiek.GestructureerdeIdentificator))
  addLiteral(kbo_uri, generiek.lokaleIdentificator, 'KBO_CKB_cleansed', XSD.string)

  naam_uri = concept_uri(lblod + 'gestructureerdeIdentificator/', str(row['Titel']))
  g.add((naam_uri, RDF.type, generiek.GestructureerdeIdentificator))
  addLiteral(naam_uri, generiek.lokaleIdentificator, 'Titel', XSD.string)

  g.add((s, rov.orgStatus, concept_uri(os, status)))
  g.add((s, org.classification, centraal_bestuur_van_de_eredienst))

  #Vestiging
  vestiging_uri = concept_uri(lblod + 'vestiging/', str(identifier))
  g.add((vestiging_uri, RDF.type, org.Site))
  g.add((s, org.hasPrimarySite, vestiging_uri))

  address_uri = concept_uri(lblod + 'adresvoorstelling/', identifier)
  g.add((address_uri, RDF.type, locn.Address))
  g.add((vestiging_uri, organisatie.bestaatUit, address_uri))
  

  #TODO ADDRESS

  #mail
  contact_uri = concept_uri(lblod + 'contactinfo/', str(identifier))
  g.add((contact_uri, RDF.type, schema.ContactPoint))
  g.add((vestiging_uri, organisatie.contactinfo, contact_uri))


  # all mails - no differentiation between secretaris and voorzitter in data --> added as general contact
  for mail in re.findall(r'[\w\.-]+@[\w\.-]+', str(row['Mail_CKB'])):
    g.add((contact_uri, schema.email, Literal(mail, datatype=XSD.string)))
  
  #TODO: cleansing
  addLiteral(address_uri, locn.thoroughfare, 'Straat_CKB')
  addLiteral(address_uri, adres['Adresvoorstelling.huisnummer'], 'Huisnr_CKB_Cleansed', datatype=XSD.string)
  addLiteral(address_uri, adres['Adresvoorstelling.busnummer'], 'Busnummer_CKB_Cleansed', datatype=XSD.string)
  addLiteral(address_uri, locn.postCode, 'Postcode_CKB', datatype=XSD.string)
  addLiteral(address_uri, adres.gemeentenaam, 'Gemeente_CKB')
  addLiteral(address_uri, locn.adminUnitL2, 'Provincie_CKB')
  g.add((address_uri, adres.land, Literal('België', lang='nl')))

  #Bestuur
  bestuur = concept_uri(lblod + 'bestuursorgaan/', row['organization_id'])
  g.add((bestuur, RDF.type, besluit.Bestuursorgaan))
  g.add((bestuur, besluit.bestuurt, s))

  bestuur_temporary = concept_uri(lblod + 'bestuursorgaan/', (row['organization_id'] + str(datetime.datetime.now().year)))
  g.add((bestuur_temporary, RDF.type, besluit.Bestuursorgaan))
  g.add((bestuur_temporary, generiek.isTijdspecialisatieVan, bestuur))

  # Voorzitter
  voorzitter = concept_uri(lblod + 'persoon/', (row['Naam_Voorzitter_CKB_first'] + row['Naam_Voorzitter_CKB_last']))
  g.add((voorzitter, RDF.type, person.Person))
  addLiteral(voorzitter, persoon.gebruikteVoornaam, 'Naam_Voorzitter_CKB_first', datatype=XSD.string)
  addLiteral(voorzitter, foaf.familyName, 'Naam_Voorzitter_CKB_last', datatype=XSD.string)

  ## Tel voorzitter - Correct properties??? 
  voorzitter_vestiging_uri = concept_uri(lblod + 'vestiging/', (str(identifier) + (row['Naam_Voorzitter_CKB_first'] + row['Naam_Voorzitter_CKB_last'])))
  g.add((voorzitter_vestiging_uri, RDF.type, org.Site))
  g.add((voorzitter, org.basedAt, voorzitter_vestiging_uri))

  voorzitter_contact_uri = concept_uri(lblod + 'contactinfo/', str(identifier+ (row['Naam_Voorzitter_CKB_first'] + row['Naam_Voorzitter_CKB_last'])))
  g.add((voorzitter_contact_uri, RDF.type, schema.ContactPoint))
  g.add((voorzitter_vestiging_uri, organisatie.contactinfo, voorzitter_contact_uri))
  addLiteral(voorzitter_contact_uri, schema.telephone, 'Tel_CKB_voorzitter_1', datatype=XSD.string)
  addLiteral(voorzitter_contact_uri, schema.telephone, 'Tel_CKB_voorzitter_2', datatype=XSD.string)


  ## Mandataris
  voorzitter_mandataris = concept_uri(lblod + 'mandataris/', (row['Naam_Voorzitter_CKB_first'] + row['Naam_Voorzitter_CKB_last'] + row['organization_id'] + 'voorzitter'))
  g.add((voorzitter_mandataris, RDF.type, mandaat.Mandataris))
  g.add((voorzitter, mandaat.isAangesteldAls, voorzitter_mandataris))
  g.add((voorzitter_mandataris, mandaat.isBestuurlijkeAliasVan, voorzitter))
  #start
  #einde
  #status ~ cf loket lokale besturen PoC https://poc-form-builder.relance.s.redpencil.io/codelijsten
  voorzitter_mandaat = concept_uri(lblod + 'mandaat/', (row['Naam_Voorzitter_CKB_first'] + row['Naam_Voorzitter_CKB_last'] + row['organization_id']))
  g.add((voorzitter_mandaat, RDF.type, mandaat.Mandaat))
  g.add((voorzitter_mandataris, org.holds, voorzitter_mandaat))
  g.add((voorzitter_mandaat, org.role, voorzitter_concept))


  g.add((bestuur_temporary, org.hasPost, voorzitter_mandaat))
  g.add((voorzitter_mandaat, org.holds, bestuur_temporary))
  
  #Secretaris
  secretaris =  concept_uri(lblod + 'persoon/', (row['Naam_secretaris_CKB_first'] + row['Naam_secretaris_CKB_last']))
  g.add((secretaris, RDF.type, person.Person))
  addLiteral(secretaris, persoon.gebruikteVoornaam, 'Naam_secretaris_CKB_first', datatype=XSD.string)
  addLiteral(secretaris, foaf.familyName, 'Naam_secretaris_CKB_last', datatype=XSD.string)
  ## Tel secretaris
  secretaris_vestiging_uri = concept_uri(lblod + 'vestiging/', (str(identifier) + (row['Naam_secretaris_CKB_first'] + row['Naam_secretaris_CKB_last'])))
  g.add((secretaris_vestiging_uri, RDF.type, org.Site))
  g.add((secretaris, org.basedAt, secretaris_vestiging_uri))

  secretaris_contact_uri = concept_uri(lblod + 'contactinfo/', str(identifier+ (row['Naam_secretaris_CKB_first'] + row['Naam_secretaris_CKB_last'])))
  g.add((secretaris_contact_uri, RDF.type, schema.ContactPoint))
  g.add((secretaris_vestiging_uri, organisatie.contactinfo, secretaris_contact_uri))
  addLiteral(secretaris_contact_uri, schema.telephone, 'Tel_CKB_secretaris_1', datatype=XSD.string)
  addLiteral(secretaris_contact_uri, schema.telephone, 'Tel_CKB_secretaris_2', datatype=XSD.string)

  #Mandataris
  secretaris_mandataris = concept_uri(lblod + 'mandataris/', (row['Naam_secretaris_CKB_first'] + row['Naam_secretaris_CKB_last'] + row['organization_id'] + 'secretaris'))
  g.add((secretaris_mandataris, RDF.type, mandaat.Mandataris))
  g.add((secretaris, mandaat.isAangesteldAls, secretaris_mandataris))
  g.add((secretaris_mandataris, mandaat.isBestuurlijkeAliasVan, secretaris))
  #start
  #einde
  #status
  secretaris_mandaat = concept_uri(lblod + 'mandaat/', (row['Naam_secretaris_CKB_first'] + row['Naam_secretaris_CKB_last'] + row['organization_id']))
  g.add((secretaris_mandaat, RDF.type, mandaat.Mandaat))
  g.add((secretaris_mandataris, org.holds, secretaris_mandaat))
  g.add((secretaris_mandaat, org.role, secretaris_concept))

  g.add((bestuur_temporary, org.hasPost, secretaris_mandaat))
  g.add((secretaris_mandaat, org.holds, bestuur_temporary))




In [ ]:
#print(g.serialize(format='turtle').decode('UTF-8'))

In [ ]:
g.serialize('ckb-qa.ttl',format='turtle')

In [ ]:
ckb.to_excel('centrale_kerkbesturen_feedback.xlsx')

# Checks/tests

In [ ]:
f = Graph();
for index, row in ckb.iterrows():
 f.add((URIRef('http://ad.hoc/s'), URIRef('http://ad.hoc/p'),Literal(row['Tel_CKB_voorzitter_1'])))

In [ ]:
print(f.serialize(format='turtle').decode('UTF-8'))

In [ ]:
ckb['Status_CKB'].unique()

In [ ]:
hash('a')

In [ ]:
import random

In [ ]:
rd = random.Random()

In [ ]:
rd.getrandbits(128)

In [ ]:
import hashlib
import uuid

m = hashlib.md5()
m.update("aaa".encode('utf-8'))
new_uuid = uuid.UUID(m.hexdigest())


# Oslo

In [ ]:
#oslo = pd.read_excel('OSLO_Mapping_20210322.xlsx', sheet_name='vocabulary_CH-core_FORMAT-Nordi')

In [ ]:
oslo 

In [ ]:
o = Graph();

oslo['Uri'] = oslo['Uri'].str.replace('org:', 'http://www.w3.org/ns/org#')

for index, row in oslo.dropna(subset=['Uri']).iterrows():
  s = URIRef(row['Uri'])
  o.add((s, RDF.type, URIRef('http://www.w3.org/2002/07/owl#ObjectProperty')))
  o.add((s, RDFS.comment, Literal(row['comment'], lang='nl')))
  o.add((s, RDFS.label, Literal(row['label'], lang='nl')))
  
print(o.serialize(format='turtle').decode('UTF-8'))

In [ ]:
o.serialize('oslo.ttl',format='turtle')

# Open issues/questions


*   identifier: not every entity has a (valid) KBO number. cf cleansinig. Uri's are currently based on internal identifier
*   heeftvesitiging ipv adres? https://data.vlaanderen.be/ns/organisatie (hasSite)
*   Base uri's
*   Mandaat --> Positie?
*   Begin/einde?
*   Secretaris ~ hoedanigheid bugdethouder?? <> penningmeester
*   Concept codes to be defined?:
**  Functionaris status 





In [ ]:
from google.colab import output
output.clear()